In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

import sys

# setting path
sys.path.append('..')

from evaluation import get_time_step_fleet
from utils import load_solution

In [2]:
solution = load_solution("../data/solution_example.json")
df = get_time_step_fleet(solution=solution, ts=1)

In [3]:
df.columns

Index(['datacenter_id', 'server_generation', 'server_id', 'action'], dtype='object')

In [4]:
from utils import load_problem_data
demand, datacenters, servers, selling_prices = load_problem_data('../data')

In [5]:
demand_selected = demand[demand['time_step'] == 1]
demand_selected

,time_step,latency_sensitivity,CPU.S1,CPU.S2,CPU.S3,CPU.S4,GPU.S1,GPU.S2,GPU.S3
0,1,high,4000,0,0,0,30,0,0
168,1,medium,6000,0,0,0,10,0,0
336,1,low,10000,0,0,0,10,0,0


In [6]:
datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


In [7]:
merged_servers_df = pd.merge(selling_prices, servers, on='server_generation')

# Calculate Total Revenue (assuming selling price represents revenue per time unit over life expectancy)
merged_servers_df['total_revenue'] = merged_servers_df['capacity'] * merged_servers_df['selling_price'] * merged_servers_df['life_expectancy']

# Calculate Total Costs
merged_servers_df['total_cost'] = (
    merged_servers_df['purchase_price'] +
    (merged_servers_df['average_maintenance_fee'] * merged_servers_df['life_expectancy']) +
    merged_servers_df['cost_of_moving']
)

# Calculate Profitability
merged_servers_df['profitability'] = merged_servers_df['total_revenue'] - merged_servers_df['total_cost']

# Sort by Profitability
most_profitable_servers = merged_servers_df.sort_values(by='profitability', ascending=False)

most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
20,GPU.S3,high,2700.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,2073600.0,456680,1616920.0
13,GPU.S3,medium,2450.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1881600.0,456680,1424920.0
6,GPU.S3,low,2150.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1651200.0,456680,1194520.0
19,GPU.S2,high,2000.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1536000.0,399720,1136280.0
18,GPU.S1,high,1875.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1440000.0,342760,1097240.0
12,GPU.S2,medium,1800.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1382400.0,399720,982680.0
11,GPU.S1,medium,1680.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1290240.0,342760,947480.0
5,GPU.S2,low,1600.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1228800.0,399720,829080.0
4,GPU.S1,low,1500.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1152000.0,342760,809240.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,460800.0,63608,397192.0


In [8]:
most_profitable_servers

,server_generation,latency_sensitivity,selling_price,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee,total_revenue,total_cost,profitability
20,GPU.S3,high,2700.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,2073600.0,456680,1616920.0
13,GPU.S3,medium,2450.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1881600.0,456680,1424920.0
6,GPU.S3,low,2150.0,GPU,"[97,168]",160000,4,4200,8,96,1000,3080,1651200.0,456680,1194520.0
19,GPU.S2,high,2000.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1536000.0,399720,1136280.0
18,GPU.S1,high,1875.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1440000.0,342760,1097240.0
12,GPU.S2,medium,1800.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1382400.0,399720,982680.0
11,GPU.S1,medium,1680.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1290240.0,342760,947480.0
5,GPU.S2,low,1600.0,GPU,"[49,120]",140000,4,3000,8,96,1000,2695,1228800.0,399720,829080.0
4,GPU.S1,low,1500.0,GPU,"[1,72]",120000,4,3000,8,96,1000,2310,1152000.0,342760,809240.0
17,CPU.S4,high,30.0,CPU,"[109,168]",22000,2,920,160,96,1000,423,460800.0,63608,397192.0


In [9]:
import uuid


def generate_random_fleet(datacenters=datacenters, servers=servers, num_servers=100, max_time_step=168):
    # Create lists of valid datacenter IDs and server generations
    datacenter_ids = datacenters['datacenter_id'].tolist()
    server_generations = servers['server_generation'].tolist()

    # Generate random data
    random_fleet = pd.DataFrame({
        'datacenter_id': np.random.choice(datacenter_ids, num_servers),
        'server_generation': np.random.choice(server_generations, num_servers),
        'server_id': [str(uuid.uuid4()) for _ in range(num_servers)],
        'time_step_of_purchase': np.random.randint(1, max_time_step + 1, num_servers)
    })

    return random_fleet

In [10]:
fleet = generate_random_fleet()
fleet

,datacenter_id,server_generation,server_id,time_step_of_purchase
0,DC1,GPU.S2,973ab9c7-cf74-44f4-ad24-51e67ee2fb00,8
1,DC3,CPU.S1,a3d4bcb5-f23b-4681-ad19-65c3542761e7,18
2,DC3,CPU.S1,f7f34c81-0cd1-47f6-afbf-eddf4333c948,20
3,DC3,GPU.S1,010a704a-2a9e-4cef-9eb5-5786e2d6e644,131
4,DC4,CPU.S1,8dcd3680-602d-48c7-b14b-e523f0495f52,23
...,...,...,...,...
95,DC3,CPU.S4,1b80871b-75d8-4099-b6c5-1be50415588f,47
96,DC2,GPU.S3,743a6107-2ba3-4269-bd94-1a7c753fd0e4,61
97,DC4,CPU.S2,a8d20830-f9bd-497d-b4a8-e5e37158fd6c,155
98,DC3,CPU.S4,221e08b0-646f-4499-b5b6-bf6b69049584,134


In [11]:
server_counts = fleet.groupby(['datacenter_id', 'server_generation']).size().reset_index(name="count")
server_counts

,datacenter_id,server_generation,count
0,DC1,CPU.S1,4
1,DC1,CPU.S2,1
2,DC1,CPU.S3,6
3,DC1,CPU.S4,2
4,DC1,GPU.S1,2
5,DC1,GPU.S2,2
6,DC1,GPU.S3,5
7,DC2,CPU.S1,4
8,DC2,CPU.S2,6
9,DC2,CPU.S3,3


In [12]:
fleet.loc[fleet["datacenter_id"] == "DC4"]

,datacenter_id,server_generation,server_id,time_step_of_purchase
4,DC4,CPU.S1,8dcd3680-602d-48c7-b14b-e523f0495f52,23
26,DC4,GPU.S3,44a2d27a-4863-4ec5-8eab-aa76bf15db77,134
35,DC4,GPU.S1,e55c5c13-98b5-49d3-9168-a902c2716afe,123
38,DC4,GPU.S3,16e0925c-534f-4489-98fc-5ff50d52de60,18
39,DC4,CPU.S4,68c55281-7b05-4767-8b36-1a74d7ec9238,101
40,DC4,CPU.S3,458cf462-4bf7-48a4-bc1c-9148b2ce29c9,38
47,DC4,CPU.S3,408e5664-0b07-4414-8ad1-baa45678866c,149
54,DC4,CPU.S1,a43a4d7f-2392-440d-b0fa-1626ead22be1,37
56,DC4,GPU.S1,5602ee44-ef67-4a8a-8bd4-7ccff597cdd5,41
58,DC4,GPU.S1,42b26f0b-4239-46b1-afa9-4180cce3f0b6,102


In [13]:
def get_server_ages(fleet, time_step):
    return fleet['time_step_of_purchase'].apply(lambda x: time_step - x)

In [14]:
get_server_ages(fleet, 168)

0     160
1     150
2     148
3      37
4     145
     ... 
95    121
96    107
97     13
98     34
99      8
Name: time_step_of_purchase, Length: 100, dtype: int64